In [ ]:
import dlib
import cv2
import sys
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

main = tk.Tk()
main.title("Final_Free Project")

cvFrame = tk.Frame(main)
cvFrame.grid(row = 0, column = 0, columnspan=6, padx = 10, pady = 10)

lbl1 = tk.Label(cvFrame)
lbl1.grid(row = 0, column = 0, columnspan=6)


click = 0

faceDetector = dlib.get_frontal_face_detector()
landmarkDetector = dlib.shape_predictor('free_project/shape_predictor_68_face_landmarks.dat')
face_cascade = cv2.CascadeClassifier('free_project/haarcascade_frontalface_alt.xml') 


def Mosaic(image):
    faceRects = faceDetector(image, 0)
    for i in range(0, len(faceRects)):
        # draw rect on detected face
        x1 = faceRects[i].left()
        y1 = faceRects[i].top()
        x2 = faceRects[i].right()
        y2 = faceRects[i].bottom()
        
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
        
    return image
    
def Sketch(image):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_gray_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    canny_edges = cv2.Canny(img_gray_blur, 15, 70) 
    # Do an invert binarize the image 
    ret, mask = cv2.threshold(canny_edges, 70, 255, cv2.THRESH_BINARY_INV)
    
    return mask

def Supreme(image):
    sun_image = cv2.imread('supreme_logo.png')
    h, w = image.shape[:2]
    
    image_gray = cv2.cvtColor(sun_image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(image_gray, 200, 255, cv2.THRESH_BINARY_INV)
    mask = sun_image
    mask_image = cv2.bitwise_and(sun_image, mask)
    sh, sw = mask_image.shape[:2]

    zero_mask = np.zeros_like(image)
    zero_mask_image = np.zeros_like(image)

    rx_x = np.int(sw*0.15); lx_x = np.int(sw*(1-0.13))
    eye_y = np.int(sh*0.45)
    eyedot = [(rx_x, eye_y), (lx_x, eye_y)]
    
    faceRects = faceDetector(image, 0)
    for i in range(0, len(faceRects)):
        landmarks = landmarkDetector(image, faceRects[i])
        rightp = (landmarks.part(36).x, landmarks.part(36).y) # right eye
        leftp = (landmarks.part(45).x, landmarks.part(45).y) # left eye
        eyedotDst = [rightp, leftp]

        ret = cv2.estimateAffinePartial2D(np.array([eyedot]), np.array([eyedotDst]))
        xform = ret[0]

        xformed_mask = cv2.warpAffine(mask, xform, (w,h))
        xformed_mask_img = cv2.warpAffine(mask_image, xform, (w, h))

        zero_mask = zero_mask + xformed_mask
        zero_mask_image = zero_mask_image + xformed_mask_img
        
    image_blend = (image*0.0 + zero_mask_image*1.0).astype(np.uint8)
    zero_mask_float = zero_mask / 255
    zero_mask_invert = 1 - zero_mask
    image_object = (image_blend * zero_mask_float).astype(np.uint8)
    image_bg = (image * (1-zero_mask_float)).astype(np.uint8)
    image_final = image_object + image_bg
    
    return image_final

def faceSupreme():
    global click; click = 3
    
def faceMosaic():
    global click; click = 2
    
def SketchEffect():
    global click; click = 1

    
btn1 = tk.Button(cvFrame, text = "Sketchify", font = ('Arial', '20', 'bold'), foreground="Black", command = SketchEffect)
btn1.grid(row = 1, column = 0, sticky = tk.N + tk.S + tk.W + tk.E)

btn2 = tk.Button(cvFrame, text = "Face Mosaic", font = ('Arial', '20', 'bold'), foreground="Green", command = faceMosaic)
btn2.grid(row = 1, column = 1, sticky = tk.N + tk.S + tk.W + tk.E)

btn3 = tk.Button(cvFrame, text = "Supreme", font = ('Arial', '20', 'bold'), foreground="Red", command = faceSupreme)
btn3.grid(row = 1, column = 2, sticky = tk.N + tk.S + tk.W + tk.E)



cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('frame_size = ', frame_size)

def show_frame():
    ret, frame = cap.read() 
    frame = cv2.flip(frame, 1) 
        
    if click == 0: 
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)        
    elif click == 1:
        frame = Sketch(frame) 
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)
    elif click == 2:
        frame = Mosaic(frame)
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)
    elif click == 3:
        frame = Supreme(frame)
        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        img = Image.fromarray(cv2image)

        imgtk = ImageTk.PhotoImage(image = img)

        lbl1.imgtk = imgtk
        lbl1.configure(image = imgtk)   
    
    main.after(10, show_frame)
    
show_frame()
main.mainloop()

In [ ]:
import dlib
import cv2 as cv
import numpy as np

detector = dlib.get_frontal_face_detector()
 
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


cap = cv.VideoCapture(0)


# range는 끝값이 포함안됨   
ALL = list(range(0, 68)) 
RIGHT_EYEBROW = list(range(17, 22))  
LEFT_EYEBROW = list(range(22, 27))  
RIGHT_EYE = list(range(36, 42))  
LEFT_EYE = list(range(42, 48))  
NOSE = list(range(27, 36))  
MOUTH_OUTLINE = list(range(48, 61))  
MOUTH_INNER = list(range(61, 68)) 
JAWLINE = list(range(0, 17)) 

index = ALL

while True:

    ret, img_frame = cap.read()

    img_gray = cv.cvtColor(img_frame, cv.COLOR_BGR2GRAY)


    dets = detector(img_gray, 1)


    for face in dets:

        shape = predictor(img_frame, face) #얼굴에서 68개 점 찾기

        list_points = []
        for p in shape.parts():
            list_points.append([p.x, p.y])

        list_points = np.array(list_points)


        for i,pt in enumerate(list_points[index]):

            pt_pos = (pt[0], pt[1])
            cv.circle(img_frame, pt_pos, 2, (0, 255, 0), -1)

        
        cv.rectangle(img_frame, (face.left(), face.top()), (face.right(), face.bottom()),
            (0, 0, 255), 3)


    cv.imshow('result', img_frame)

    
    key = cv.waitKey(1)

    if key == 27:
        break
    
    elif key == ord('1'):
        index = ALL
    elif key == ord('2'):
        index = LEFT_EYEBROW + RIGHT_EYEBROW
    elif key == ord('3'):
        index = LEFT_EYE + RIGHT_EYE
    elif key == ord('4'):
        index = NOSE
    elif key == ord('5'):
        index = MOUTH_OUTLINE+MOUTH_INNER
    elif key == ord('6'):
        index = JAWLINE


cap.release()